# TRANSLATE ENGLISH HATECHECK TO TARGET LANGUAGES

## Target languages:
- Chinese (DeepL)
- Dutch (DeepL)
- French (DeepL)
- German (DeepL)
- Italian (DeepL)
- Polish (DeepL)
- Portuguese (DeepL)
- Spanish (DeepL)
- Hindi (Google)
- Arabic (Google)

## Load Relevant Packages

In [1]:
import os
import requests
import json
import time
import pandas as pd

from google.cloud import translate

## Load English HateCheck

In [2]:
hatecheck_df = pd.read_csv('0_data/test_suite_cases.csv', index_col=0)
hatecheck_df.head()

,functionality,case_id,test_case,label_gold,target_ident,direction,focus_words,focus_lemma,ref_case_id,ref_templ_id,templ_id,case_templ
0,derog_neg_emote_h,1,I hate women.,hateful,women,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
1,derog_neg_emote_h,2,I hate trans people.,hateful,trans people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
2,derog_neg_emote_h,3,I hate gay people.,hateful,gay people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
3,derog_neg_emote_h,4,I hate black people.,hateful,black people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
4,derog_neg_emote_h,5,I hate disabled people.,hateful,disabled people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].


## Initiate DeepL Translator

In [3]:
deepl_url = 'https://api.deepl.com/v2/translate'

with open("./z_keys/deepl-translate-key-071221.json") as f:
    deepl_auth_key = json.load(f)['key']

def trans_deepl(text, target_lang):
    
     # remove special tokens because they can mess up translation
    text = text.replace(" [URL]", "").replace("[URL]", "").replace(" [USER]","").replace("[USER]","")
    text = text.replace("RT: ", "").replace("RT:", "")
    text = text.strip()
    
    payload = {"target_lang": target_lang, "auth_key": deepl_auth_key, "text": text}
    response = requests.post(url = deepl_url, data = payload)
    
    for translation in response.json()["translations"]:
        trans_text = translation["text"]   
    
    return trans_text

In [4]:
%%time
# test case
res = trans_deepl('[USER] This is a bandaid!', "de")
res

CPU times: user 18.7 ms, sys: 5.6 ms, total: 24.3 ms
Wall time: 546 ms


'Das ist ein Pflaster!'

## Initiate Google Translator

In [8]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./z_keys/google-translate-key-071221.json"
client = translate.TranslationServiceClient()

In [9]:
project_id = "hatecheck-translations"
parent = f"projects/{project_id}"

def trans_google(text, target_lang):
    
    response = client.translate_text(
        contents = [text],
        target_language_code = target_lang,
        parent=parent
    )
    
    for translation in response.translations:
        trans_text = translation.translated_text   
    
    return trans_text

In [10]:
%%time
# test case
res = trans_google('[USER] This is a bandaid!', "de")
res

CPU times: user 5.3 ms, sys: 2.88 ms, total: 8.18 ms
Wall time: 229 ms


'[USER] Das ist ein Pflaster!'